In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from NeuralNetworksTutorialNetClass import Net

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 7.2519e-02,  1.5867e-01,  2.9775e-02, -9.3486e-02, -2.1724e-01,
         -9.2994e-02,  6.3263e-05,  6.4214e-02,  1.3944e-01,  1.3357e-01]],
       grad_fn=<AddmmBackward>)


In [6]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [7]:
output = net(input)
target = torch.randn(10)
target = target.view(1,-1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.6467, grad_fn=<MseLossBackward>)


In [8]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

In [9]:
net.zero_grad()
print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0025,  0.0058,  0.0019, -0.0020, -0.0096,  0.0121])


In [10]:
learning_rate = .01
for f in net.parameters():
    # weight = weight - learning_rate * gradient
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = .01)
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()